### Modelos baseados em árvores

<br>

Ao longo do módulo, discutimos bastante as árvores de decisão, bem como ensemble de árvores, como Random Forest e algoritmos do tipo boosting.

Esses __ensembles acabam tendo muitos hiperparâmetros;__ escolhe-los de forma manual acaba sendo muito custoso e tedioso. 

Neste exercício, vamos discutir a respeito da metolodia __grid-search__, que otimiza essa busca de hiperparâmetros.

Considere o dataset abaixo (basta executar as células):

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

In [3]:
#problema de regressão

X, y = load_diabetes().data, load_diabetes().target
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.25, random_state = 42)
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

(331, 10) (111, 10) (331,) (111,)


Imagine que queremos testar - usando cross-validation - várias instâncias de Random Forests: com 10 árvores, com 100 árvores, com 1000 árvores, com profundidade máxima 1, 5, 10. 

Como podemos proceder? O código abaixo exemplifica um jeito:

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

for n_est in [10,100,100]:
    for prof in [1,5,10]:
        rf = RandomForestRegressor(n_estimators=n_est, max_depth=prof)
        cvres = cross_val_score(estimator=rf, X = Xtrain, y = ytrain, cv = 3, scoring='r2')
        print("estimators: ", n_est, " prof: ", prof, " | R2 mean / std: ", cvres.mean(), ' / ', cvres.std())

estimators:  10  prof:  1  | R2 mean / std:  0.31668666439282617  /  0.05192198772248859
estimators:  10  prof:  5  | R2 mean / std:  0.3880261560163776  /  0.050459713075200426
estimators:  10  prof:  10  | R2 mean / std:  0.40705792888610476  /  0.025851167514224
estimators:  100  prof:  1  | R2 mean / std:  0.3243564415884665  /  0.04519494762188192
estimators:  100  prof:  5  | R2 mean / std:  0.42554077495428855  /  0.04300018691060576
estimators:  100  prof:  10  | R2 mean / std:  0.41235675612525435  /  0.04309041104116178
estimators:  100  prof:  1  | R2 mean / std:  0.33925844150098206  /  0.03641927954666465
estimators:  100  prof:  5  | R2 mean / std:  0.4258495683693086  /  0.04659020204051035
estimators:  100  prof:  10  | R2 mean / std:  0.4165109138808491  /  0.041539344109688466


Podemos, com algum trabalho, escolher o melhor modelo.

Se quisermos testar mais parâmetros, podemos aumentar nosso loop... mais isso vai ficando cada vez mais complicado.

A proposta do __grid-search__ é justamente fazer isso de forma mais automática!

Podemos importar a função GridSearchCV do módulo model_selection do sklearn e usá-la para isso. 
Na prática, precisamos definir um __estimador base__ para o grid. Além disso, precisamos definir um __dicionário de parâmetros__ a ser testado. Ainda, definiremos a quantidade de folds para cross-validation e qual a métrica de performance que queremos otimizar:

In [5]:
#importando a função
from sklearn.model_selection import GridSearchCV

In [6]:
#definindo o estimador base
estimador_base = RandomForestRegressor()

#definindo o dicionario de parâmetros do modelo
params_RF = {"n_estimators":[10,1000], "max_depth":[2,10]}

In [7]:
grid = GridSearchCV(estimator = estimador_base, 
                    param_grid = params_RF, 
                    scoring = 'r2', 
                    cv = 3)

grid

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

In [8]:
#treinando os modelos no grid
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

O objeto "grid", após o treinamento acima, conterá várias informações muito relevantes. 

__1- "best_params_":__ retorna os melhores parâmetros, de acordo com a métrica de performance avaliada na cross-validation;

__1- "best_score_":__ retorna o melhor score - métrica de performance - nos dados de validação;

__1- "best_estimator_":__ retorna o melhor modelo, já treinado;

__1- "cv_results_":__ retorna uma visão geral dos resultados.

In [9]:
grid.best_params_

{'max_depth': 10, 'n_estimators': 1000}

In [10]:
grid.best_score_

0.4219897115096661

In [11]:
grid.best_estimator_

RandomForestRegressor(max_depth=10, n_estimators=1000)

In [12]:
grid.cv_results_

{'mean_fit_time': array([0.00998505, 0.90091777, 0.01296377, 1.29852859]),
 'std_fit_time': array([1.39705060e-05, 5.25020624e-03, 1.87730977e-06, 1.31364411e-02]),
 'mean_score_time': array([0.00099699, 0.0542024 , 0.00099683, 0.06249793]),
 'std_score_time': array([1.10692885e-06, 9.55878063e-04, 8.48537942e-07, 1.69367413e-03]),
 'param_max_depth': masked_array(data=[2, 2, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[10, 1000, 10, 1000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'n_estimators': 10},
  {'max_depth': 2, 'n_estimators': 1000},
  {'max_depth': 10, 'n_estimators': 10},
  {'max_depth': 10, 'n_estimators': 1000}],
 'split0_test_score': array([0.3260469 , 0.350742  , 0.35176998, 0.36385111]),
 'split1_test_score': array([0.44118386, 0.44451014, 0.40535785, 0.45928804]),
 'split2_tes

__Exercício 1:__ Utilizando o dataset abaixo, faça um grid_search com KNN's, Random Forests e GradientBoostings e retorne o melhor modelo de cada tipo.

__Obs.:__ Lembre-se de fazer um pré-processamento nos dados!

In [13]:
#preco_mediano_das_casas é a variável target
df = pd.read_csv("preco_casas.csv")
print(df.shape)
df.head()

(20640, 10)


,longitude,latitude,idade_mediana_das_casas,total_comodos,total_quartos,populacao,familias,salario_mediano,preco_mediano_das_casas,proximidade_ao_mar
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,PERTO DA BAÍA
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,PERTO DA BAÍA
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,PERTO DA BAÍA
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,PERTO DA BAÍA
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,PERTO DA BAÍA


In [14]:
df.isnull().sum()

longitude                    0
latitude                     0
idade_mediana_das_casas      0
total_comodos                0
total_quartos              207
populacao                    0
familias                     0
salario_mediano              0
preco_mediano_das_casas      0
proximidade_ao_mar           0
dtype: int64

In [15]:
df.dtypes

longitude                  float64
latitude                   float64
idade_mediana_das_casas    float64
total_comodos              float64
total_quartos              float64
populacao                  float64
familias                   float64
salario_mediano            float64
preco_mediano_das_casas    float64
proximidade_ao_mar          object
dtype: object

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [17]:
dftrain, dftest = train_test_split(df, test_size=0.25, random_state=0)
print (df.shape)
print(dftrain.shape)
print (dftest.shape)

(20640, 10)
(15480, 10)
(5160, 10)


In [18]:
# definição da função de pré-processamento dos dados

def pre_processamento (df, dataset_de_treino = True, cat_encoder = None, std_scaler = None):
    dff = df.copy()
        
    # criação de novas variáveis
    dff["pessoas_por_familia"] = dff["populacao"] / dff["familias"]
    dff["comodos_por_pessoa"] = dff["total_comodos"] / dff["populacao"]
    dff["quartos_por_comodos"] = dff["total_quartos"] / dff["total_comodos"]
    
    # retirar valores faltantes
    dff = dff.dropna(axis = 0)
    
    colsnum = ['idade_mediana_das_casas','total_comodos','total_quartos','populacao','familias','salario_mediano',
              'pessoas_por_familia','comodos_por_pessoa','quartos_por_comodos']
    colscat = ['proximidade_ao_mar']
        
    if dataset_de_treino: # no dataset de treino, o modelo será treinado antes de transformar os dados
        
        encoder = OneHotEncoder() # OHE para as variáveis categóricas
        variaveis_ohe = encoder.fit_transform(dff[colscat]).toarray()
        
        std_sc = StandardScaler() # adotamos o StandardScaler para as variáveis numéricas contínuas
        variaveis_std = std_sc.fit_transform(dff[colsnum])
        
        X, y = np.c_[variaveis_ohe, variaveis_std], dff.preco_mediano_das_casas.values
        return X, y, encoder, std_sc
    
    else: # no dataset de teste, os dados serão transformados a partir do modelo já treinado 
        variaveis_ohe = cat_encoder.transform(dff[colscat]).toarray()
        
        variaveis_std = std_scaler.transform(dff[colsnum])
        
        X, y = np.c_[variaveis_ohe, variaveis_std], dff.preco_mediano_das_casas.values
        return X, y
    

In [19]:
# pre-processamento dos dados de treino
Xtrain, ytrain, encoder_train, scaler_train = pre_processamento(dftrain, dataset_de_treino = True,
                                                                cat_encoder = None, std_scaler = None)

In [20]:
# pré-processamentos dos dados de teste
Xtest, ytest = pre_processamento(dftest, dataset_de_treino = False,
                                cat_encoder = encoder_train, std_scaler = scaler_train)

In [21]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [22]:
# Grid Search com algoritmo KNN

estimador_base = KNeighborsRegressor()
params_KNN = {'n_neighbors':[5,10,20,30]}
grid_KNN = GridSearchCV (estimator = estimador_base, param_grid = params_KNN, scoring = 'r2', cv = 5)
grid_KNN.fit(Xtrain, ytrain)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [5, 10, 20, 30]}, scoring='r2')

In [23]:
grid_KNN.best_params_

{'n_neighbors': 10}

In [24]:
grid_KNN.best_score_

0.7133474231558996

In [25]:
import time

In [26]:
# Grid Search com algoritmo Random Forest
t0 = time.time()

estimador_base = RandomForestRegressor()
params_RF = {"n_estimators":[50,100,200], "max_depth":[6,10,20],"max_features":["auto"]}
grid_RF = GridSearchCV (estimator = estimador_base, param_grid = params_RF, scoring = 'r2', cv = 3)
grid_RF.fit(Xtrain, ytrain)

print ("Tempo decorrido (min): ", (time.time()-t0)/60)

Tempo decorrido (min):  2.44579515059789


In [27]:
grid_RF.best_params_

{'max_depth': 20, 'max_features': 'auto', 'n_estimators': 200}

In [28]:
grid_RF.best_score_

0.7393903553942023

In [29]:
grid_RF.cv_results_

{'mean_fit_time': array([ 1.24338341,  2.60902603,  4.93116204,  1.85272344,  3.75694609,
         7.44775526,  2.97239749,  5.67215379, 11.36928884]),
 'std_fit_time': array([0.01480665, 0.10367502, 0.02390023, 0.00330546, 0.03208791,
        0.02082876, 0.06718193, 0.03793444, 0.02706318]),
 'mean_score_time': array([0.01624926, 0.03091447, 0.05982828, 0.02625259, 0.05087741,
        0.10271454, 0.05086652, 0.09574564, 0.19113302]),
 'std_score_time': array([4.62898798e-04, 8.44958815e-04, 1.25300249e-05, 9.32946946e-04,
        8.26855668e-04, 4.23914693e-03, 2.93323422e-03, 7.81202510e-04,
        1.68396547e-03]),
 'param_max_depth': masked_array(data=[6, 6, 6, 10, 10, 10, 20, 20, 20],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_max_features': masked_array(data=['auto', 'auto', 'auto', 'auto', 'auto', 'auto', 'auto',
                    'auto', 'auto'],
         

In [30]:
# Grid Search com algoritmo Gradient Boosting
t0 = time.time()

estimador_base = GradientBoostingRegressor()
params_GB = {"learning_rate":[0.05,0.1,0.2] ,"n_estimators":[50,100,200], "max_depth":[3,5,7]}
grid_GB = GridSearchCV (estimator = estimador_base, param_grid = params_GB, scoring = 'r2', cv = 3)
grid_GB.fit(Xtrain, ytrain)

print ("Tempo decorrido (min): ", (time.time()-t0)/60)

Tempo decorrido (min):  5.850344630082448


In [31]:
grid_GB.best_params_

{'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 200}

In [32]:
grid_GB.best_score_

0.7449669519088044

In [33]:
grid_GB.cv_results_

{'mean_fit_time': array([ 1.0897433 ,  2.14026388,  4.38161635,  1.73534489,  3.50461706,
         7.06313721,  2.37998414,  4.75597556,  9.66417424,  1.12168328,
         2.21441126,  4.41386588,  2.02592921,  3.58274484,  7.07942001,
         2.40822895,  4.89257574, 10.15384396,  1.08744836,  2.19478901,
         4.40555501,  1.76263491,  4.00760825,  7.21671844,  2.46508026,
         5.06279945, 10.25226935]),
 'std_fit_time': array([0.00863277, 0.01925936, 0.05786298, 0.01174344, 0.04879391,
        0.08191777, 0.05315639, 0.07005339, 0.09047711, 0.05831721,
        0.06924826, 0.08045878, 0.20193936, 0.11447328, 0.08632925,
        0.06047629, 0.09826459, 0.31952864, 0.00262534, 0.0196305 ,
        0.09666776, 0.00728592, 0.43596221, 0.11065708, 0.07715775,
        0.12100111, 0.33338317]),
 'mean_score_time': array([0.00534415, 0.00831437, 0.01463095, 0.00832796, 0.0159843 ,
        0.02357999, 0.01396092, 0.02360892, 0.03589209, 0.00465059,
        0.00731389, 0.01263269, 0.008

__Exercício 2:__ Crie uma classe para comparar o grid_search dentre vários modelos distintos.
    
    
Essa classe, gridSearchAll(), já está pré-desenvolvida no código abaixo. O exercício consiste de __completar essa classe.__ Para isso, crie o métodos fit_all, que irá treinar, usando grid_search, todos os grids que tenham sido pré-construídos e inseridos na classe.
Ainda, a quantidade de folds para a validação cruzada no grid_search deve ser implementada no método construtor da classe, bem como qual a métrica de performance a ser avaliada. 
Finalmente, salve o melhor modelo de cada grid e tenha um método best_all_grid_models que retorna o melhor modelo dentre todos os grids.

In [34]:
class gridSearchAll():
    
    def __init__(self, num_folds, scoring):
        self.grid_models = []
        self.scoring = scoring
        self.num_folds = num_folds
            
    def insert_model(self, estimator_base, param_grid):
        self.grid_models.append([estimator_base, param_grid])
        
    def fit_all(self, X, y):
        self.bestAll = []
        for models in self.grid_models:
            grid = GridSearchCV (estimator=models[0], param_grid = models[1], scoring = self.scoring, cv = self.num_folds)
            grid.fit(X,y)
            self.bestAll.append([grid.best_estimator_, grid.best_score_])
    
    def best_all_grid_models(self):
        best_score = 0
        for i in self.bestAll:
            if i[1] > best_score:
                best_index = self.bestAll.index(i)
                best_score = i[1]
                
        self.best_estimator = self.bestAll[best_index][0]
        self.best_score = self.bestAll[best_index][1]
        
        print ("Melhor modelo: ", self.best_estimator)
        print ("Melhor score: ", self.best_score)                

In [35]:
gd = gridSearchAll(num_folds = 3, scoring = 'r2')

In [36]:
gd.grid_models

[]

In [37]:
params_RF = {"n_estimators":[10,1000], "max_depth":[2,10]}

In [38]:
gd.insert_model(estimator_base = RandomForestRegressor(), param_grid = params_RF)

In [39]:
gd.grid_models

[[RandomForestRegressor(), {'n_estimators': [10, 1000], 'max_depth': [2, 10]}]]

In [40]:
gd.insert_model(estimator_base = KNeighborsRegressor(), param_grid = {"n_neighbors":[1,2,10]})

In [41]:
gd.grid_models

[[RandomForestRegressor(), {'n_estimators': [10, 1000], 'max_depth': [2, 10]}],
 [KNeighborsRegressor(), {'n_neighbors': [1, 2, 10]}]]

In [42]:
gd.fit_all(Xtrain, ytrain)

In [43]:
gd.best_all_grid_models()

Melhor modelo:  RandomForestRegressor(max_depth=10, n_estimators=1000)
Melhor score:  0.7388682147352504


__Exercício 3:__ Usando a classe criada, analise novamente os modelos criados no exercício 1.

In [44]:
gd1 = gridSearchAll(num_folds = 3, scoring = 'r2')
params_KNN = {'n_neighbors':[5,10,20,30]}
params_RF = {"n_estimators":[50,100,200], "max_depth":[6,10,20],"max_features":["auto"]}
params_GB = {"learning_rate":[0.05,0.1,0.2] ,"n_estimators":[50,100,200], "max_depth":[3,5,7]}

gd1.insert_model(estimator_base = KNeighborsRegressor(), param_grid = params_KNN)
gd1.insert_model(estimator_base = RandomForestRegressor(), param_grid = params_RF)
gd1.insert_model(estimator_base = GradientBoostingRegressor(), param_grid = params_GB)

In [45]:
gd1.fit_all(Xtrain, ytrain)

In [46]:
gd1.best_all_grid_models()

Melhor modelo:  GradientBoostingRegressor(learning_rate=0.05, max_depth=5, n_estimators=200)
Melhor score:  0.7448811377576822
